In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
pd.options.display.max_columns = None

### Import Data

In [3]:
path = r'/Users/henning/Documents/Data Analytics Portfolio/Instacart Project - ICP'

In [4]:
df_orders = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_cleaned.pkl'))

In [5]:
df_prior_baskets = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'old_baskets_cleaned.pkl'))

In [6]:
df_recent_baskets = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'recent_baskets_cleaned.pkl'))

In [7]:
df_products = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'products_cleaned.pkl'))

In [8]:
df_department = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'departments.csv'))

In [9]:
df_aisles = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'aisles.csv'))

In [10]:
df_customer = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customers_cleaned.pkl'))

### Small data cleaning on department and aisles data set

In [11]:
# changing IDs into string type
df_department['department_id'] = df_department['department_id'].astype('str')
df_aisles['aisle_id'] = df_aisles['aisle_id'].astype('str')

### Merging Order Data Set with the old Baskets Data Set

In [12]:
# filtering df_order to include only the previous orders.
df_orders_prior = df_orders[df_orders['eval_set'] == 'prior']

In [13]:
# deciding type of join by checking number of rows
df_orders_prior.shape

(3214874, 7)

In [14]:
df_prior_baskets.shape

(32434489, 3)

In [15]:
# combining dataframes
df_prior =  df_prior_baskets.merge(df_orders_prior, on = 'order_id', how='left', indicator = True)

In [16]:
df_prior['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [17]:
# dropping _merge column for the next combination
df_prior = df_prior.drop(columns = ['_merge'])

### Merging Order Data Set with the recent Baskets Data Set

In [18]:
# filtering df_order to include only the recent orders.
df_orders_recent = df_orders[df_orders['eval_set'] == 'train']

In [19]:
# deciding type of join by checking number of rows
df_orders_recent.shape

(131209, 7)

In [20]:
df_recent_baskets.shape

(1384617, 3)

In [21]:
# combining dataframes
df_recent =  df_recent_baskets.merge(df_orders_recent, on = 'order_id', how='left', indicator = True)

In [22]:
df_recent['_merge'].value_counts()

_merge
both          1384617
left_only           0
right_only          0
Name: count, dtype: int64

In [23]:
# dropping _merge column for the next combination
df_recent = df_recent.drop(columns = ['_merge'])

### Merging df_prior and df_recent to df_1

In [24]:
# Concatenating the Data
frames = [df_prior, df_recent]
df_1 = pd.concat(frames, axis = 0)

In [25]:
df_1

,order_id,product_id,reordered,user_id,eval_set,order_number,order_weekday,order_hour_of_day,days_since_last_order
0,2,33120,1,202279,prior,3,5,9,8.0
1,2,28985,1,202279,prior,3,5,9,8.0
2,2,9327,0,202279,prior,3,5,9,8.0
3,2,45918,1,202279,prior,3,5,9,8.0
4,2,30035,0,202279,prior,3,5,9,8.0
...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,1,169679,train,30,0,10,4.0
1384613,3421063,35548,1,169679,train,30,0,10,4.0
1384614,3421070,35951,1,139822,train,15,6,10,8.0
1384615,3421070,16953,1,139822,train,15,6,10,8.0


### Merging df_1 with the next data frame: products

In [26]:
# deciding type of join by checking number of rows
df_1.shape

(33819106, 9)

In [27]:
df_products.shape

(49688, 5)

In [28]:
# combining dataframes with left join to include all orders, but to exclude unordered products.
df_2 =  df_1.merge(df_products, on = 'product_id', how='left', indicator = True)

In [29]:
df_2['_merge'].value_counts()

_merge
both          33817591
left_only         2117
right_only           0
Name: count, dtype: int64

In [30]:
df_2[df_2['_merge'] == 'left_only']

,order_id,product_id,reordered,user_id,eval_set,order_number,order_weekday,order_hour_of_day,days_since_last_order,product_name,aisle_id,department_id,prices,_merge
16863,1770,6799,1,678,prior,18,3,13,12.0,NaN,NaN,NaN,NaN,left_only
32562,3387,6799,1,175450,prior,9,1,19,17.0,NaN,NaN,NaN,NaN,left_only
66688,7099,6799,0,27,prior,63,3,10,1.0,NaN,NaN,NaN,NaN,left_only
72827,7754,6799,1,1629,prior,20,3,18,10.0,NaN,NaN,NaN,NaN,left_only
76643,8153,6799,0,56820,prior,2,1,19,30.0,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33745421,3236456,6799,1,135766,train,8,2,20,30.0,NaN,NaN,NaN,NaN,left_only
33764342,3284493,6799,1,98534,train,30,1,18,16.0,NaN,NaN,NaN,NaN,left_only
33784175,3332831,6799,0,137861,train,4,0,14,30.0,NaN,NaN,NaN,NaN,left_only
33802900,3381717,6799,0,169142,train,9,0,14,1.0,NaN,NaN,NaN,NaN,left_only


There are orders, where the data on the product is not available. The decision is to leave them in as they still can provide statistical insights on the time and date when a order is placed.

In [31]:
# dropping _merge column for the next combination
df_2 = df_2.drop(columns = ['_merge'])

### Merging df_2 with department information

In [32]:
# deciding type of join by checking number of rows
df_2.shape

(33819708, 13)

In [33]:
df_department.shape

(21, 2)

In [34]:
# combining dataframes
df_3 =  df_2.merge(df_department, on = 'department_id', how='left', indicator = True)

In [35]:
df_3['_merge'].value_counts()

_merge
both          33817591
left_only         2117
right_only           0
Name: count, dtype: int64

In [36]:
# dropping _merge column for the next combination
df_3 = df_3.drop(columns = ['_merge'])

### Merging df_3 with aisles information

In [37]:
# combining dataframes
df_4 =  df_3.merge(df_aisles, on = 'aisle_id', how='left', indicator = True)

In [38]:
df_4['_merge'].value_counts()

_merge
both          33817591
left_only         2117
right_only           0
Name: count, dtype: int64

In [39]:
# dropping _merge column for the next combination
df_4 = df_4.drop(columns = ['_merge'])

### Merging df_4 with customer data

In [40]:
# checking number of rows
df_4.shape

(33819708, 15)

In [41]:
df_customer.shape

(206209, 7)

In [42]:
# combining dataframes
df_instacart =  df_4.merge(df_customer, on = 'user_id', how='inner', indicator = True)

In [43]:
df_instacart['_merge'].value_counts()

_merge
both          33819708
left_only            0
right_only           0
Name: count, dtype: int64

In [44]:
# dropping _merge column for the next combination
df_instacart = df_instacart.drop(columns = ['_merge'])

### Removing ID columns

In [45]:
# because the eval_set and ID columns are only necessary for combining the data sets,
# they can now be removed as we don't need them for the upcoming analysis.
# Only the User_id stays, as it is the only way to identify a specific customer.
df_instacart = df_instacart.drop(columns = ['eval_set', 'order_id', 'product_id', 'aisle_id', 'department_id'])

In [46]:
# renaming user_id column for better understanding and readability
df_instacart.rename(columns = {'user_id' : 'customer'}, inplace = True)

In [47]:
# final check on data
df_instacart.sample(3)

,reordered,customer,order_number,order_weekday,order_hour_of_day,days_since_last_order,product_name,prices,department,aisle,gender,state,age,date_joined,civil_status,income
24521834,0,169044,1,6,7,NaN,Whole Vitamin D Milk,14.5,dairy eggs,milk,Female,Ohio,51,3/5/2020,married,106062
28286798,0,138143,8,4,16,27.0,Cotton Swabs,4.8,personal care,eye ear care,Male,Michigan,51,4/14/2017,married,33109
13472581,0,71645,10,2,8,27.0,Organic Greek Whole Milk Blended Vanilla Bean ...,5.9,dairy eggs,yogurt,Female,South Carolina,79,11/17/2018,married,68487


### Exporting merged Data Set

In [48]:
df_instacart.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'merged_data_set.pkl'))